In [2]:
# Import Pandas as Dependencies
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import requests
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 as ps
import pandas.io.sql as sqlio
from bs4 import BeautifulSoup as bs
from config import password
from config import protocol
from config import username
from config import host
from config import host
from config import port
from config import database_name



  # ETL

  ## Extract DataFrame 

In [3]:
# Read the post covid file and store into Pandas DataFrame
food_access_raw_df = pd.read_csv("FoodAccessResearchAtlasData2019.csv", dtype={'CensusTract': str})
food_access_raw_df.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,01001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,01001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,01001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,01001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,01001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


## Transform DataFrame

In [4]:
# Get list of columns
list(food_access_raw_df.columns)

['CensusTract',
 'State',
 'County',
 'Urban',
 'Pop2010',
 'OHU2010',
 'GroupQuartersFlag',
 'NUMGQTRS',
 'PCTGQTRS',
 'LILATracts_1And10',
 'LILATracts_halfAnd10',
 'LILATracts_1And20',
 'LILATracts_Vehicle',
 'HUNVFlag',
 'LowIncomeTracts',
 'PovertyRate',
 'MedianFamilyIncome',
 'LA1and10',
 'LAhalfand10',
 'LA1and20',
 'LATracts_half',
 'LATracts1',
 'LATracts10',
 'LATracts20',
 'LATractsVehicle_20',
 'LAPOP1_10',
 'LAPOP05_10',
 'LAPOP1_20',
 'LALOWI1_10',
 'LALOWI05_10',
 'LALOWI1_20',
 'lapophalf',
 'lapophalfshare',
 'lalowihalf',
 'lalowihalfshare',
 'lakidshalf',
 'lakidshalfshare',
 'laseniorshalf',
 'laseniorshalfshare',
 'lawhitehalf',
 'lawhitehalfshare',
 'lablackhalf',
 'lablackhalfshare',
 'laasianhalf',
 'laasianhalfshare',
 'lanhopihalf',
 'lanhopihalfshare',
 'laaianhalf',
 'laaianhalfshare',
 'laomultirhalf',
 'laomultirhalfshare',
 'lahisphalf',
 'lahisphalfshare',
 'lahunvhalf',
 'lahunvhalfshare',
 'lasnaphalf',
 'lasnaphalfshare',
 'lapop1',
 'lapop1share',
 

In [5]:
# Create a subset dataframe with selected columns
food_access_df = food_access_raw_df[["CensusTract", "State", "County", "Urban", "Pop2010", "OHU2010", "PovertyRate",
                                    "MedianFamilyIncome", "LAhalfand10", "TractLOWI", "TractKids", "TractSeniors", "TractWhite",
                                     "TractBlack", "TractAsian", "TractNHOPI", "TractAIAN", "TractOMultir",
                                     "TractHispanic", "TractHUNV", "TractSNAP"]]
food_access_df.tail()


,CensusTract,State,County,Urban,Pop2010,OHU2010,PovertyRate,MedianFamilyIncome,LAhalfand10,TractLOWI,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
72526,56043000200,Wyoming,Washakie County,0,3326,1317,9.7,67254.0,1,765.0,...,593.0,3106.0,6.0,15.0,0.0,27.0,172.0,309.0,61.0,64.0
72527,56043000301,Wyoming,Washakie County,1,2665,1154,11.6,64152.0,1,774.0,...,399.0,2377.0,5.0,23.0,0.0,40.0,220.0,446.0,88.0,41.0
72528,56043000302,Wyoming,Washakie County,1,2542,1021,16.3,69605.0,1,789.0,...,516.0,2312.0,11.0,10.0,1.0,26.0,182.0,407.0,23.0,64.0
72529,56045951100,Wyoming,Weston County,0,3314,1322,17.5,74500.0,1,955.0,...,499.0,3179.0,15.0,10.0,1.0,47.0,62.0,91.0,47.0,34.0
72530,56045951300,Wyoming,Weston County,1,3894,1699,17.3,76838.0,1,1095.0,...,650.0,3706.0,6.0,10.0,2.0,44.0,126.0,125.0,34.0,110.0


In [6]:
# Create "StateFIPS" column by slicing "CensusTract" column 
food_access_df["StateFIPS"] = food_access_df["CensusTract"].str.slice(0, 2)
food_access_df.head()


C:\Users\meena\AppData\Local\Temp\ipykernel_3492\2251358069.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_access_df["StateFIPS"] = food_access_df["CensusTract"].str.slice(0, 2)


,CensusTract,State,County,Urban,Pop2010,OHU2010,PovertyRate,MedianFamilyIncome,LAhalfand10,TractLOWI,...,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,StateFIPS
0,01001020100,Alabama,Autauga County,1,1912,693,11.3,81250.0,1,455.0,...,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0,01
1,01001020200,Alabama,Autauga County,1,2170,743,17.9,49000.0,1,802.0,...,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0,01
2,01001020300,Alabama,Autauga County,1,3373,1256,15.0,62609.0,1,1306.0,...,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0,01
3,01001020400,Alabama,Autauga County,1,4386,1722,2.8,70607.0,1,922.0,...,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0,01
4,01001020500,Alabama,Autauga County,1,10766,4082,15.2,96334.0,1,2242.0,...,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0,01


In [7]:
# Shift "StateFIPS" column to first position
first_column = food_access_df.pop("StateFIPS")
  
# insert column using insert(position,column_name,first_column) function
food_access_df.insert(0, "StateFIPS", first_column)

# Display Dataframe
food_access_df.head()

,StateFIPS,CensusTract,State,County,Urban,Pop2010,OHU2010,PovertyRate,MedianFamilyIncome,LAhalfand10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,01,01001020100,Alabama,Autauga County,1,1912,693,11.3,81250.0,1,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,01,01001020200,Alabama,Autauga County,1,2170,743,17.9,49000.0,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,01,01001020300,Alabama,Autauga County,1,3373,1256,15.0,62609.0,1,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,01,01001020400,Alabama,Autauga County,1,4386,1722,2.8,70607.0,1,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,01,01001020500,Alabama,Autauga County,1,10766,4082,15.2,96334.0,1,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


In [8]:
# Calculate percentage population of each group
food_access_df[["TractLOWI_PCT", "TractKids_PCT", "TractSeniors_PCT", "TractWhite_PCT",
                "TractBlack_PCT", "TractAsian_PCT", "TractNHOPI_PCT", "TractAIAN_PCT",
                "TractOMultir_PCT", "TractHispanic_PCT"]] = food_access_df[["TractLOWI", "TractKids", "TractSeniors", "TractWhite",
                                     "TractBlack", "TractAsian", "TractNHOPI", "TractAIAN", "TractOMultir",
                                     "TractHispanic"]]\
    .apply(lambda x: x/food_access_df['Pop2010']*100).round(2)


# Display Dataframe
food_access_df.head()

C:\Users\meena\AppData\Local\Temp\ipykernel_3492\3642112065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_access_df[["TractLOWI_PCT", "TractKids_PCT", "TractSeniors_PCT", "TractWhite_PCT",
C:\Users\meena\AppData\Local\Temp\ipykernel_3492\3642112065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_access_df[["TractLOWI_PCT", "TractKids_PCT", "TractSeniors_PCT", "TractWhite_PCT",
C:\Users\meena\AppData\Local\Temp\ipykernel_3492\3642112065.py:2: SettingWithCopyWarning: 
A value is trying to 

,StateFIPS,CensusTract,State,County,Urban,Pop2010,OHU2010,PovertyRate,MedianFamilyIncome,LAhalfand10,...,TractLOWI_PCT,TractKids_PCT,TractSeniors_PCT,TractWhite_PCT,TractBlack_PCT,TractAsian_PCT,TractNHOPI_PCT,TractAIAN_PCT,TractOMultir_PCT,TractHispanic_PCT
0,01,01001020100,Alabama,Autauga County,1,1912,693,11.3,81250.0,1,...,23.80,26.52,11.56,84.83,11.35,0.73,0.00,0.73,2.35,2.30
1,01,01001020200,Alabama,Autauga County,1,2170,743,17.9,49000.0,1,...,36.96,27.93,9.86,40.92,56.08,0.23,0.00,0.23,2.53,3.46
2,01,01001020300,Alabama,Autauga County,1,3373,1256,15.0,62609.0,1,...,38.72,26.50,13.02,76.37,19.18,0.50,0.15,0.33,3.47,2.58
3,01,01001020400,Alabama,Autauga County,1,4386,1722,2.8,70607.0,1,...,21.02,23.14,20.61,93.16,4.40,0.41,0.09,0.25,1.69,1.94
4,01,01001020500,Alabama,Autauga County,1,10766,4082,15.2,96334.0,1,...,20.82,29.37,10.46,80.49,13.35,2.75,0.08,0.45,2.88,3.30


In [9]:
# Calculate percentage TractHUNV and TractSNAP  group
food_access_df[["TractHUNV_PCT", "TractSNAP_PCT"]] = food_access_df[["TractHUNV", "TractSNAP"]]\
    .apply(lambda x: x/food_access_df['OHU2010']*100).round(2)


# Display Dataframe
food_access_df.head()

C:\Users\meena\AppData\Local\Temp\ipykernel_3492\3074993017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_access_df[["TractHUNV_PCT", "TractSNAP_PCT"]] = food_access_df[["TractHUNV", "TractSNAP"]]\
C:\Users\meena\AppData\Local\Temp\ipykernel_3492\3074993017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_access_df[["TractHUNV_PCT", "TractSNAP_PCT"]] = food_access_df[["TractHUNV", "TractSNAP"]]\


,StateFIPS,CensusTract,State,County,Urban,Pop2010,OHU2010,PovertyRate,MedianFamilyIncome,LAhalfand10,...,TractSeniors_PCT,TractWhite_PCT,TractBlack_PCT,TractAsian_PCT,TractNHOPI_PCT,TractAIAN_PCT,TractOMultir_PCT,TractHispanic_PCT,TractHUNV_PCT,TractSNAP_PCT
0,01,01001020100,Alabama,Autauga County,1,1912,693,11.3,81250.0,1,...,11.56,84.83,11.35,0.73,0.00,0.73,2.35,2.30,0.87,14.72
1,01,01001020200,Alabama,Autauga County,1,2170,743,17.9,49000.0,1,...,9.86,40.92,56.08,0.23,0.00,0.23,2.53,3.46,11.98,21.00
2,01,01001020300,Alabama,Autauga County,1,3373,1256,15.0,62609.0,1,...,13.02,76.37,19.18,0.50,0.15,0.33,3.47,2.58,7.88,13.69
3,01,01001020400,Alabama,Autauga County,1,4386,1722,2.8,70607.0,1,...,20.61,93.16,4.40,0.41,0.09,0.25,1.69,1.94,1.22,5.69
4,01,01001020500,Alabama,Autauga County,1,10766,4082,15.2,96334.0,1,...,10.46,80.49,13.35,2.75,0.08,0.45,2.88,3.30,5.63,8.30


In [10]:
# Get dataframe information
food_access_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72531 entries, 0 to 72530
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   StateFIPS           72531 non-null  object 
 1   CensusTract         72531 non-null  object 
 2   State               72531 non-null  object 
 3   County              72531 non-null  object 
 4   Urban               72531 non-null  int64  
 5   Pop2010             72531 non-null  int64  
 6   OHU2010             72531 non-null  int64  
 7   PovertyRate         72528 non-null  float64
 8   MedianFamilyIncome  71783 non-null  float64
 9   LAhalfand10         72531 non-null  int64  
 10  TractLOWI           72527 non-null  float64
 11  TractKids           72527 non-null  float64
 12  TractSeniors        72527 non-null  float64
 13  TractWhite          72527 non-null  float64
 14  TractBlack          72527 non-null  float64
 15  TractAsian          72527 non-null  float64
 16  Trac

In [11]:
# Get NaN values for each column
food_access_df.isnull().sum()

StateFIPS               0
CensusTract             0
State                   0
County                  0
Urban                   0
Pop2010                 0
OHU2010                 0
PovertyRate             3
MedianFamilyIncome    748
LAhalfand10             0
TractLOWI               4
TractKids               4
TractSeniors            4
TractWhite              4
TractBlack              4
TractAsian              4
TractNHOPI              4
TractAIAN               4
TractOMultir            4
TractHispanic           4
TractHUNV               4
TractSNAP               4
TractLOWI_PCT           4
TractKids_PCT           4
TractSeniors_PCT        4
TractWhite_PCT          4
TractBlack_PCT          4
TractAsian_PCT          4
TractNHOPI_PCT          4
TractAIAN_PCT           4
TractOMultir_PCT        4
TractHispanic_PCT       4
TractHUNV_PCT         110
TractSNAP_PCT         109
dtype: int64

In [12]:
# Drop rows with NAN values
food_access_df = food_access_df.dropna()
 
# Reset the index
food_access_df = food_access_df.reset_index(drop = True)

# Display Dataframe
food_access_df.head()

,StateFIPS,CensusTract,State,County,Urban,Pop2010,OHU2010,PovertyRate,MedianFamilyIncome,LAhalfand10,...,TractSeniors_PCT,TractWhite_PCT,TractBlack_PCT,TractAsian_PCT,TractNHOPI_PCT,TractAIAN_PCT,TractOMultir_PCT,TractHispanic_PCT,TractHUNV_PCT,TractSNAP_PCT
0,01,01001020100,Alabama,Autauga County,1,1912,693,11.3,81250.0,1,...,11.56,84.83,11.35,0.73,0.00,0.73,2.35,2.30,0.87,14.72
1,01,01001020200,Alabama,Autauga County,1,2170,743,17.9,49000.0,1,...,9.86,40.92,56.08,0.23,0.00,0.23,2.53,3.46,11.98,21.00
2,01,01001020300,Alabama,Autauga County,1,3373,1256,15.0,62609.0,1,...,13.02,76.37,19.18,0.50,0.15,0.33,3.47,2.58,7.88,13.69
3,01,01001020400,Alabama,Autauga County,1,4386,1722,2.8,70607.0,1,...,20.61,93.16,4.40,0.41,0.09,0.25,1.69,1.94,1.22,5.69
4,01,01001020500,Alabama,Autauga County,1,10766,4082,15.2,96334.0,1,...,10.46,80.49,13.35,2.75,0.08,0.45,2.88,3.30,5.63,8.30


In [13]:
# Confirm NaN values removal from Dataframe
food_access_df.isnull().sum()

StateFIPS             0
CensusTract           0
State                 0
County                0
Urban                 0
Pop2010               0
OHU2010               0
PovertyRate           0
MedianFamilyIncome    0
LAhalfand10           0
TractLOWI             0
TractKids             0
TractSeniors          0
TractWhite            0
TractBlack            0
TractAsian            0
TractNHOPI            0
TractAIAN             0
TractOMultir          0
TractHispanic         0
TractHUNV             0
TractSNAP             0
TractLOWI_PCT         0
TractKids_PCT         0
TractSeniors_PCT      0
TractWhite_PCT        0
TractBlack_PCT        0
TractAsian_PCT        0
TractNHOPI_PCT        0
TractAIAN_PCT         0
TractOMultir_PCT      0
TractHispanic_PCT     0
TractHUNV_PCT         0
TractSNAP_PCT         0
dtype: int64

In [14]:
# Get dataframe information
food_access_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71782 entries, 0 to 71781
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   StateFIPS           71782 non-null  object 
 1   CensusTract         71782 non-null  object 
 2   State               71782 non-null  object 
 3   County              71782 non-null  object 
 4   Urban               71782 non-null  int64  
 5   Pop2010             71782 non-null  int64  
 6   OHU2010             71782 non-null  int64  
 7   PovertyRate         71782 non-null  float64
 8   MedianFamilyIncome  71782 non-null  float64
 9   LAhalfand10         71782 non-null  int64  
 10  TractLOWI           71782 non-null  float64
 11  TractKids           71782 non-null  float64
 12  TractSeniors        71782 non-null  float64
 13  TractWhite          71782 non-null  float64
 14  TractBlack          71782 non-null  float64
 15  TractAsian          71782 non-null  float64
 16  Trac

In [15]:
# Check for duplicate values in the df 
food_access_df.duplicated().sum()

0

In [16]:
# describe dataframe
food_access_df.describe()

,Urban,Pop2010,OHU2010,PovertyRate,MedianFamilyIncome,LAhalfand10,TractLOWI,TractKids,TractSeniors,TractWhite,...,TractSeniors_PCT,TractWhite_PCT,TractBlack_PCT,TractAsian_PCT,TractNHOPI_PCT,TractAIAN_PCT,TractOMultir_PCT,TractHispanic_PCT,TractHUNV_PCT,TractSNAP_PCT
count,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,...,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000,71782.000000
mean,0.760065,4279.328300,1621.995333,15.080299,77038.372949,0.683277,1394.443983,1032.096069,560.069627,3101.745842,...,13.677105,71.998286,13.719088,4.395257,0.161650,1.012340,8.713386,15.277930,9.464745,13.519311
std,0.427046,1939.251049,714.857476,11.513766,37544.385035,0.465202,979.193659,611.062326,349.906067,1788.110051,...,7.237739,25.632149,22.191989,8.385029,0.922285,4.636075,10.003063,20.833944,14.108777,12.248158
min,0.000000,24.000000,7.000000,0.000000,2499.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2922.000000,1118.000000,6.500000,51484.500000,0.000000,691.000000,621.000000,325.000000,1872.000000,...,9.150000,58.550000,1.090000,0.480000,0.000000,0.190000,2.340000,2.420000,2.520000,4.690000
50%,1.000000,4027.000000,1533.000000,12.000000,68821.000000,1.000000,1172.000000,930.000000,500.000000,2929.000000,...,12.960000,80.940000,3.950000,1.470000,0.030000,0.390000,4.740000,6.200000,5.300000,10.170000
75%,1.000000,5341.000000,2027.000000,20.500000,93870.750000,1.000000,1853.000000,1318.000000,720.000000,4130.000000,...,16.740000,92.220000,14.630000,4.300000,0.110000,0.770000,10.620000,18.080000,10.930000,18.760000
max,1.000000,37452.000000,16043.000000,99.500000,250001.000000,1.000000,12562.000000,11845.000000,17271.000000,28983.000000,...,89.370000,100.000000,100.000000,89.850000,73.840000,98.890000,81.150000,99.390000,1594.870000,420.370000


In [17]:
# Save the dataframe
food_access_df.to_csv("food_access.csv")

## Load DataFrame to pgAdmin

In [ ]:
# Read the cleaned up csv file and load to dataframe.
food_access_df = pd.read_csv("food_access.csv", index_col=[0], dtype={'StateFIPS': str})

# Convert CensusTract to String
food_access_df['CensusTract'] = food_access_df["CensusTract"].astype(str)

# Read the dataframe
food_access_df.head()

In [ ]:
# Confirm dtypes
food_access_df.dtypes

In [ ]:
# Connection to PostgresSQL
protocol = protocol
username = username
password = password
host = host
port = port
database_name = database_name
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [ ]:
# Confirm tables names
engine.table_names()

In [ ]:
# Export the transformed data into the PostgresSQL
food_access_df.to_sql(name='food_access', con=engine, if_exists='replace', index=False)

In [ ]:
# Read the data from Postgres table
pd.read_sql_query('select * from food_access', con=engine).head()